# Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
from ml_core.normalization import z_score_normalization
from ml_core.data_process import *
from ml_core.regression import GradientDescent, OrdinaryLeastSquares, StochasticGradientDescent
from sklearn.model_selection import train_test_split
from ml_core.metrics.error_metrics import *
from ml_core.metrics import error_metric_in_history_models
from ml_core.metrics.error_metrics import *
from ml_core.classification import LogisticBinaryModel,LinearRegressionMulticlass
from ml_core.classification import StatisticalClassifiers, GaussianDiscriminant, NaiveBayesGaussian
from sklearn.naive_bayes import GaussianNB

# Leitura dos Dados

In [2]:
df = pd.read_csv('data/breastcancer.csv', header=None)

In [3]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0.0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0.0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0.0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0.0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0.0


# Preparar os dados para experimentos


In [4]:
arr = df.to_numpy()

In [5]:
X = arr[:, :30]
Y = arr[:, -1]

In [6]:
X_norm = np.empty_like(X)

for idx, col in enumerate(X.T):
    
    norm = z_score_normalization(col)
    
    X_norm[:, idx] = norm

In [7]:
y_norm = Y.reshape([-1, 1])

In [8]:
X_ones = add_ones_column(X_norm)

In [9]:
X_trn, X_tst, y_trn, y_tst = train_test_split(X_ones, y_norm, test_size=0.2, random_state=42)

# Classificação Binária

## Regressão Logistica

In [10]:
first_model = LogisticBinaryModel.first_model(
    lenght=X_trn.shape[1],
    fill_value=1
)

In [11]:
gd = GradientDescent(ephocs=1000)

In [12]:
gd_model = gd.fit(X_trn, y_trn, first_model=first_model.__copy__())

  0%|          | 0/1000 [00:00<?, ?it/s]

In [13]:
preds = gd_model.predict_ajust(X_tst)

In [14]:
acuracy(y_tst, preds), f1_score(y_tst, preds), precision(y_tst, preds)

(0.956140350877193, 0.7777777777777777, 0.6422018348623854)

## Analise Discriminante Gaussiano

In [16]:
X_trn_ = X_trn[:, 1:]  
y_trn_dummies = get_dummies(y_trn.reshape([-1]))

In [17]:
gaussian_discriminant = StatisticalClassifiers(model=GaussianDiscriminant)

In [18]:
model_gaussian = gaussian_discriminant.fit(X_trn_, y_trn_dummies)

In [19]:
preds = model_gaussian.predict(X_tst[:, 1:])

In [22]:
acuracy(y_tst, preds), f1_score(y_tst, preds), precision(y_tst, preds)

(0.956140350877193, 0.7555555555555555, 0.6238532110091743)

## Naive Bayes

In [25]:
naive_bayes = StatisticalClassifiers(model=NaiveBayesGaussian)

In [26]:
model_naive = naive_bayes.fit(X_trn_, y_trn_dummies)

In [27]:
preds = model_naive.predict(X_tst[:, 1:])

In [29]:
acuracy(y_tst, preds), f1_score(y_tst, preds), precision(y_tst, preds)

(0.9649122807017544, 0.7734806629834254, 0.6363636363636364)